# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [84]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [ ]:
stocks=pd.read_csv('sp_500_stocks.csv')
from secretstest import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [ ]:
symbol='AAPL'
api_url=f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data=requests.get(api_url).json()


## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [87]:
data['year1ChangePercent']

0.38073188344833075

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [88]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [89]:
final_dataframe=pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings :
    batch_api_call_url=f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe=final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ],
        index=my_columns),
            ignore_index=True
        )
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,164.760,0.573052,N/A
1,AAL,20.280,0.766851,N/A
2,AAP,233.870,0.565055,N/A
3,AAPL,151.460,0.391115,N/A
4,ABBV,119.720,0.421356,N/A
5,ABC,128.700,0.302063,N/A
6,ABMD,365.520,0.395588,N/A
7,ABT,132.460,0.246577,N/A
8,ACN,366.400,0.68022,N/A
9,ADBE,663.350,0.44781,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [90]:
final_dataframe.sort_values('One-Year Price Return',ascending=False,inplace=True)
final_dataframe=final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,148,DVN,43.10,3.97383,N/A
1,175,FANG,115.88,3.46687,N/A
2,317,MRO,16.93,3.44493,N/A
3,357,OXY,35.83,2.89345,N/A
4,275,LB,82.95,2.28505,N/A
5,299,MCHP,82.50,2.03142,N/A
6,111,COTY,9.11,2.00795,N/A
7,195,FTNT,331.70,1.9374,N/A
8,160,EOG,95.20,1.81474,N/A
9,23,ALB,264.89,1.77418,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [91]:
def portfolio_input():
    global portfolio_size
    portfolio_size=input('Enter size of your portfolio:')
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number')
        portfolio_size=input('Enter size of your portfolio:')
portfolio_input()
print(portfolio_size)

Enter size of your portfolio:10000000
10000000


In [92]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

C:\Users\MSI\Anaconda3.4\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,148,DVN,43.10,3.97383,4640
1,175,FANG,115.88,3.46687,1725
2,317,MRO,16.93,3.44493,11813
3,357,OXY,35.83,2.89345,5581
4,275,LB,82.95,2.28505,2411
5,299,MCHP,82.50,2.03142,2424
6,111,COTY,9.11,2.00795,21953
7,195,FTNT,331.70,1.9374,602
8,160,EOG,95.20,1.81474,2100
9,23,ALB,264.89,1.77418,755


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [121]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [122]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,163.820,N/A,0.554148,N/A,0.182297,N/A,0.0272122,N/A,0.00848799,N/A,N/A
1,AAL,20.000,N/A,0.753984,N/A,-0.0939295,N/A,-0.0303508,N/A,-0.0900106,N/A,N/A
2,AAP,237.940,N/A,0.549946,N/A,0.139391,N/A,0.0686786,N/A,0.0832105,N/A,N/A
3,AAPL,152.680,N/A,0.377664,N/A,0.139805,N/A,0.0232899,N/A,0.044697,N/A,N/A
4,ABBV,120.770,N/A,0.430893,N/A,0.0557001,N/A,-0.000851287,N/A,0.0680774,N/A,N/A
5,ABC,131.100,N/A,0.307455,N/A,0.025304,N/A,0.0090497,N/A,0.0260547,N/A,N/A
6,ABMD,355.280,N/A,0.390743,N/A,0.0931889,N/A,0.0704415,N/A,0.0842387,N/A,N/A
7,ABT,127.920,N/A,0.255916,N/A,0.0827763,N/A,0.0702615,N/A,0.10711,N/A,N/A
8,ACN,372.800,N/A,0.680213,N/A,0.236287,N/A,0.123815,N/A,0.0984353,N/A,N/A
9,ADBE,669.030,N/A,0.437232,N/A,0.260877,N/A,0.0298885,N/A,0.112533,N/A,N/A


In [123]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

pd.set_option("display.max_rows", None, "display.max_columns", None)
hqm_dataframe.replace(to_replace='None', value=0, inplace=True)
for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = score(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

C:\Users\MSI\Anaconda3.4\lib\site-packages\numpy\core\fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\MSI\Anaconda3.4\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [124]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,163.820,N/A,0.554148,0.609901,0.182297,0.748515,0.027212,0.487129,0.008488,0.269307,N/A
1,AAL,20.000,N/A,0.753984,0.764356,-0.093929,0.146535,-0.030351,0.273267,-0.090011,0.0356436,N/A
2,AAP,237.940,N/A,0.549946,0.605941,0.139391,0.669307,0.068679,0.641584,0.083210,0.730693,N/A
3,AAPL,152.680,N/A,0.377664,0.431683,0.139805,0.671287,0.023290,0.463366,0.044697,0.489109,N/A
4,ABBV,120.770,N/A,0.430893,0.50297,0.055700,0.49505,-0.000851,0.386139,0.068077,0.637624,N/A
5,ABC,131.100,N/A,0.307455,0.336634,0.025304,0.411881,0.009050,0.411881,0.026055,0.370297,N/A
6,ABMD,355.280,N/A,0.390743,0.449505,0.093189,0.578218,0.070441,0.655446,0.084239,0.740594,N/A
7,ABT,127.920,N/A,0.255916,0.29703,0.082776,0.552475,0.070261,0.653465,0.107110,0.833663,N/A
8,ACN,372.800,N/A,0.680213,0.732673,0.236287,0.849505,0.123815,0.786139,0.098435,0.80396,N/A
9,ADBE,669.030,N/A,0.437232,0.512871,0.260877,0.875248,0.029888,0.50297,0.112533,0.847525,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [125]:
from statistics import mean #mean([2,6])

for row in hqm_dataframe.index:
    momentum_percentiles=[]
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row,f'{time_period} Return Percentile'])
    
    hqm_dataframe.loc[row,'HQM Score']= mean(momentum_percentiles)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,163.820,N/A,0.554148,0.609901,0.182297,0.748515,0.027212,0.487129,0.008488,0.269307,0.528713
1,AAL,20.000,N/A,0.753984,0.764356,-0.093929,0.146535,-0.030351,0.273267,-0.090011,0.0356436,0.30495
2,AAP,237.940,N/A,0.549946,0.605941,0.139391,0.669307,0.068679,0.641584,0.083210,0.730693,0.661881
3,AAPL,152.680,N/A,0.377664,0.431683,0.139805,0.671287,0.023290,0.463366,0.044697,0.489109,0.513861
4,ABBV,120.770,N/A,0.430893,0.50297,0.055700,0.49505,-0.000851,0.386139,0.068077,0.637624,0.505446
5,ABC,131.100,N/A,0.307455,0.336634,0.025304,0.411881,0.009050,0.411881,0.026055,0.370297,0.382673
6,ABMD,355.280,N/A,0.390743,0.449505,0.093189,0.578218,0.070441,0.655446,0.084239,0.740594,0.605941
7,ABT,127.920,N/A,0.255916,0.29703,0.082776,0.552475,0.070261,0.653465,0.107110,0.833663,0.584158
8,ACN,372.800,N/A,0.680213,0.732673,0.236287,0.849505,0.123815,0.786139,0.098435,0.80396,0.793069
9,ADBE,669.030,N/A,0.437232,0.512871,0.260877,0.875248,0.029888,0.50297,0.112533,0.847525,0.684653


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [127]:
hqm_dataframe.sort_values('HQM Score', ascending=False,inplace=True)
hqm_dataframe=hqm_dataframe[:50]
hqm_dataframe.reset_index(drop=True,inplace=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MRO,17.560,N/A,3.468462,0.988119,0.528368,0.972277,0.480309,0.988119,0.202198,0.976238,0.981188
1,DVN,43.900,N/A,3.881401,0.992079,0.824318,0.990099,0.655265,0.990099,0.143170,0.916832,0.972277
2,F,18.810,N/A,1.349355,0.948515,0.564091,0.978218,0.288963,0.952475,0.272018,0.992079,0.967822
3,NVDA,272.140,N/A,1.072194,0.90099,0.747786,0.986139,0.334274,0.970297,0.251065,0.990099,0.961881
4,ALB,268.000,N/A,1.803885,0.974257,0.551132,0.974257,0.252281,0.938614,0.169661,0.950495,0.959406
5,FANG,118.040,N/A,3.580778,0.990099,0.387522,0.958416,0.464059,0.984158,0.134800,0.89703,0.957426
6,OXY,34.580,N/A,2.915351,0.986139,0.387270,0.956436,0.342654,0.974257,0.118298,0.869307,0.946535
7,ODFL,356.330,N/A,0.854377,0.837624,0.350815,0.942574,0.292578,0.956436,0.217268,0.978218,0.928713
8,EOG,96.370,N/A,1.834980,0.976238,0.291630,0.90297,0.289433,0.954455,0.121022,0.875248,0.927228
9,HAL,25.930,N/A,1.162913,0.922772,0.325929,0.922772,0.240852,0.928713,0.155454,0.932673,0.926733


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [115]:
portfolio_input()

Enter size of your portfolio:1000000


In [128]:
position_size=float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i,'Number of Shares to Buy']=math.floor(position_size/hqm_dataframe.loc[i,'Price'])
    
hqm_dataframe

C:\Users\MSI\Anaconda3.4\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MRO,17.560,1138,3.468462,0.988119,0.528368,0.972277,0.480309,0.988119,0.202198,0.976238,0.981188
1,DVN,43.900,455,3.881401,0.992079,0.824318,0.990099,0.655265,0.990099,0.143170,0.916832,0.972277
2,F,18.810,1063,1.349355,0.948515,0.564091,0.978218,0.288963,0.952475,0.272018,0.992079,0.967822
3,NVDA,272.140,73,1.072194,0.90099,0.747786,0.986139,0.334274,0.970297,0.251065,0.990099,0.961881
4,ALB,268.000,74,1.803885,0.974257,0.551132,0.974257,0.252281,0.938614,0.169661,0.950495,0.959406
5,FANG,118.040,169,3.580778,0.990099,0.387522,0.958416,0.464059,0.984158,0.134800,0.89703,0.957426
6,OXY,34.580,578,2.915351,0.986139,0.387270,0.956436,0.342654,0.974257,0.118298,0.869307,0.946535
7,ODFL,356.330,56,0.854377,0.837624,0.350815,0.942574,0.292578,0.956436,0.217268,0.978218,0.928713
8,EOG,96.370,207,1.834980,0.976238,0.291630,0.90297,0.289433,0.954455,0.121022,0.875248,0.927228
9,HAL,25.930,771,1.162913,0.922772,0.325929,0.922772,0.240852,0.928713,0.155454,0.932673,0.926733


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [129]:
writer=pd.ExcelWriter('momentum_strategy.xlsx',engine='xlsxwriter')
hqm_dataframe.to_excel(writer,sheet_name="Momentum Strategy",index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [130]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [131]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [132]:
writer.save()